# Hospitality: Delta Liquid Clustering Demo


## Overview


This notebook demonstrates the power of **Delta Liquid Clustering** in Oracle AI Data Platform (AIDP) Workbench using a hospitality and tourism analytics use case. Liquid clustering automatically optimizes data layout for query performance without requiring manual partitioning or Z-Ordering.

### What is Liquid Clustering?

Liquid clustering automatically identifies and groups similar data together based on clustering columns you define. This optimization happens automatically during data ingestion and maintenance operations, providing:

- **Automatic optimization**: No manual tuning required
- **Improved query performance**: Faster queries on clustered columns
- **Reduced maintenance**: No need for manual repartitioning
- **Adaptive clustering**: Adjusts as data patterns change

### Use Case: Hotel Guest Experience and Revenue Management

We'll analyze hotel booking and guest experience data. Our clustering strategy will optimize for:

- **Guest-specific queries**: Fast lookups by guest ID
- **Time-based analysis**: Efficient filtering by booking and stay dates
- **Revenue patterns**: Quick aggregation by room type and booking channels

### AIDP Environment Setup

This notebook leverages the existing Spark session in your AIDP environment.

In [ ]:
# Create hospitality catalog and analytics schema

# In AIDP, catalogs provide data isolation and governance

spark.sql("CREATE CATALOG IF NOT EXISTS hospitality")

spark.sql("CREATE SCHEMA IF NOT EXISTS hospitality.analytics")

print("Hospitality catalog and analytics schema created successfully!")

Hospitality catalog and analytics schema created successfully!


## Step 2: Create Delta Table with Liquid Clustering

### Table Design

Our `guest_stays` table will store:

- **guest_id**: Unique guest identifier
- **booking_date**: Date booking was made
- **check_in_date**: Guest arrival date
- **room_type**: Type of room booked
- **booking_channel**: How booking was made (OTA, Direct, etc.)
- **total_revenue**: Total booking revenue
- **guest_satisfaction**: Guest satisfaction score (1-10)

### Clustering Strategy

We'll cluster by `guest_id` and `booking_date` because:

- **guest_id**: Guests often make multiple bookings, grouping their stay history together
- **booking_date**: Time-based queries are critical for revenue analysis, seasonal trends, and booking patterns
- This combination optimizes for both guest relationship management and temporal revenue analytics

In [ ]:
# Create Delta table with liquid clustering

# CLUSTER BY defines the columns for automatic optimization

spark.sql("""

CREATE TABLE IF NOT EXISTS hospitality.analytics.guest_stays (

    guest_id STRING,

    booking_date DATE,

    check_in_date DATE,

    room_type STRING,

    booking_channel STRING,

    total_revenue DECIMAL(8,2),

    guest_satisfaction INT

)

USING DELTA

CLUSTER BY (guest_id, booking_date)

""")

print("Delta table with liquid clustering created successfully!")

print("Clustering will automatically optimize data layout for queries on guest_id and booking_date.")

Delta table with liquid clustering created successfully!
Clustering will automatically optimize data layout for queries on guest_id and booking_date.


## Step 3: Generate Hospitality Sample Data

### Data Generation Strategy

We'll create realistic hotel booking and guest data including:

- **5,000 guests** with multiple bookings over time
- **Room types**: Standard, Deluxe, Suite, Executive
- **Booking channels**: Direct, Online Travel Agency, Corporate, Walk-in
- **Seasonal patterns**: Peak seasons, weekend vs weekday pricing

### Why This Data Pattern?

This data simulates real hospitality scenarios where:

- Guest loyalty programs require historical booking tracking
- Revenue management depends on booking channel analysis
- Seasonal pricing strategies drive occupancy optimization
- Guest satisfaction impacts reputation and repeat business
- Channel performance requires continuous monitoring

In [ ]:
# Generate sample hospitality guest booking data

# Using fully qualified imports to avoid conflicts

import random

from datetime import datetime, timedelta


# Define hospitality data constants

ROOM_TYPES = ['Standard', 'Deluxe', 'Suite', 'Executive']

BOOKING_CHANNELS = ['Direct', 'Online Travel Agency', 'Corporate', 'Walk-in']

# Base revenue parameters by room type

REVENUE_PARAMS = {

    'Standard': {'base_rate': 120, 'satisfaction': 7.8},

    'Deluxe': {'base_rate': 200, 'satisfaction': 8.2},

    'Suite': {'base_rate': 350, 'satisfaction': 8.8},

    'Executive': {'base_rate': 280, 'satisfaction': 8.5}

}

# Channel margins (affect final revenue)

CHANNEL_MARGINS = {

    'Direct': 1.0,

    'Online Travel Agency': 0.85,

    'Corporate': 0.90,

    'Walk-in': 0.95

}


# Generate guest booking records

booking_data = []

base_date = datetime(2024, 1, 1)


# Create 5,000 guests with 2-8 bookings each

for guest_num in range(1, 5001):

    guest_id = f"GST{guest_num:06d}"
    
    # Each guest gets 2-8 bookings over 12 months

    num_bookings = random.randint(2, 8)
    
    for i in range(num_bookings):

        # Spread bookings over 12 months

        days_offset = random.randint(0, 365)

        booking_date = base_date + timedelta(days=days_offset)
        
        # Check-in date (usually within 1-30 days of booking)

        checkin_offset = random.randint(1, 30)

        check_in_date = booking_date + timedelta(days=checkin_offset)
        
        # Select room type

        room_type = random.choice(ROOM_TYPES)

        params = REVENUE_PARAMS[room_type]
        
        # Select booking channel

        booking_channel = random.choice(BOOKING_CHANNELS)

        channel_margin = CHANNEL_MARGINS[booking_channel]
        
        # Calculate revenue with variations

        # Seasonal pricing (higher in peak season)

        month = check_in_date.month

        if month in [6, 7, 8]:  # Summer peak

            seasonal_factor = 1.3

        elif month in [11, 12]:  # Holiday season

            seasonal_factor = 1.4

        else:

            seasonal_factor = 1.0
        
        # Weekend pricing

        if check_in_date.weekday() >= 5:  # Saturday = 5, Sunday = 6

            weekend_factor = 1.2

        else:

            weekend_factor = 1.0
        
        # Stay length (1-7 nights)

        stay_length = random.randint(1, 7)
        
        # Calculate total revenue

        revenue_variation = random.uniform(0.9, 1.1)

        total_revenue = round(params['base_rate'] * stay_length * seasonal_factor * weekend_factor * channel_margin * revenue_variation, 2)
        
        # Guest satisfaction (varies by room type and some randomness)

        satisfaction_variation = random.randint(-2, 2)

        guest_satisfaction = max(1, min(10, params['satisfaction'] + satisfaction_variation))
        
        booking_data.append({

            "guest_id": guest_id,

            "booking_date": booking_date.date(),

            "check_in_date": check_in_date.date(),

            "room_type": room_type,

            "booking_channel": booking_channel,

            "total_revenue": float(total_revenue),

            "guest_satisfaction": int(guest_satisfaction)

        })



print(f"Generated {len(booking_data)} guest booking records")

print("Sample record:", booking_data[0])

Generated 24925 guest booking records
Sample record: {'guest_id': 'GST000001', 'booking_date': datetime.date(2024, 9, 11), 'check_in_date': datetime.date(2024, 10, 8), 'room_type': 'Standard', 'booking_channel': 'Online Travel Agency', 'total_revenue': 97.25, 'guest_satisfaction': 7}


## Step 4: Insert Data Using PySpark

### Data Insertion Strategy

We'll use PySpark to:

1. **Create DataFrame** from our generated data
2. **Insert into Delta table** with liquid clustering
3. **Verify the insertion** with a sample query

### Why PySpark for Insertion?

- **Distributed processing**: Handles large datasets efficiently
- **Type safety**: Ensures data integrity
- **Optimization**: Leverages Spark's query optimization
- **Liquid clustering**: Automatically applies clustering during insertion

In [ ]:
# Insert data using PySpark DataFrame operations

# Using fully qualified function references to avoid conflicts


# Create DataFrame from generated data

df_bookings = spark.createDataFrame(booking_data)


# Display schema and sample data

print("DataFrame Schema:")

df_bookings.printSchema()



print("\nSample Data:")

df_bookings.show(5)


# Insert data into Delta table with liquid clustering

# The CLUSTER BY (guest_id, booking_date) will automatically optimize the data layout

df_bookings.write.mode("overwrite").saveAsTable("hospitality.analytics.guest_stays")


print(f"\nSuccessfully inserted {df_bookings.count()} records into hospitality.analytics.guest_stays")

print("Liquid clustering automatically optimized the data layout during insertion!")

DataFrame Schema:
root
 |-- booking_channel: string (nullable = true)
 |-- booking_date: date (nullable = true)
 |-- check_in_date: date (nullable = true)
 |-- guest_id: string (nullable = true)
 |-- guest_satisfaction: long (nullable = true)
 |-- room_type: string (nullable = true)
 |-- total_revenue: double (nullable = true)


Sample Data:


+--------------------+------------+-------------+---------+------------------+---------+-------------+
|     booking_channel|booking_date|check_in_date| guest_id|guest_satisfaction|room_type|total_revenue|
+--------------------+------------+-------------+---------+------------------+---------+-------------+
|Online Travel Agency|  2024-09-11|   2024-10-08|GST000001|                 7| Standard|        97.25|
|              Direct|  2024-02-08|   2024-02-17|GST000001|                 7|    Suite|       841.19|
|              Direct|  2024-11-10|   2024-11-12|GST000001|                 6|    Suite|      2441.59|
|             Walk-in|  2024-06-16|   2024-06-25|GST000001|                 8| Standard|       983.68|
|Online Travel Agency|  2024-12-27|   2025-01-19|GST000001|                10|Executive|       586.94|
+--------------------+------------+-------------+---------+------------------+---------+-------------+
only showing top 5 rows




Successfully inserted 24925 records into hospitality.analytics.guest_stays
Liquid clustering automatically optimized the data layout during insertion!


## Step 5: Demonstrate Liquid Clustering Benefits

### Query Performance Analysis

Now let's see how liquid clustering improves query performance. We'll run queries that benefit from our clustering strategy:

1. **Guest booking history** (clustered by guest_id)
2. **Time-based revenue analysis** (clustered by booking_date)
3. **Combined guest + time queries** (optimal for our clustering)

### Expected Performance Benefits

With liquid clustering, these queries should be significantly faster because:

- **Data locality**: Related records are physically grouped together
- **Reduced I/O**: Less data needs to be read from disk
- **Automatic optimization**: No manual tuning required

In [ ]:
# Demonstrate liquid clustering benefits with optimized queries


# Query 1: Guest booking history - benefits from guest_id clustering

print("=== Query 1: Guest Booking History ===")

guest_history = spark.sql("""

SELECT guest_id, booking_date, room_type, total_revenue, guest_satisfaction

FROM hospitality.analytics.guest_stays

WHERE guest_id = 'GST000001'

ORDER BY booking_date DESC

""")



guest_history.show()

print(f"Records found: {guest_history.count()}")



# Query 2: Time-based revenue analysis - benefits from booking_date clustering

print("\n=== Query 2: Recent High-Value Bookings ===")

high_value = spark.sql("""

SELECT booking_date, guest_id, room_type, total_revenue, booking_channel

FROM hospitality.analytics.guest_stays

WHERE booking_date >= '2024-06-01' AND total_revenue > 1000

ORDER BY total_revenue DESC, booking_date DESC

""")



high_value.show()

print(f"High-value bookings found: {high_value.count()}")



# Query 3: Combined guest + time query - optimal for our clustering strategy

print("\n=== Query 3: Guest Spending Trends ===")

spending_trends = spark.sql("""

SELECT guest_id, booking_date, room_type, total_revenue, guest_satisfaction

FROM hospitality.analytics.guest_stays

WHERE guest_id LIKE 'GST000%' AND booking_date >= '2024-04-01'

ORDER BY guest_id, booking_date

""")



spending_trends.show()

print(f"Spending trend records found: {spending_trends.count()}")

=== Query 1: Guest Booking History ===


+---------+------------+---------+-------------+------------------+
| guest_id|booking_date|room_type|total_revenue|guest_satisfaction|
+---------+------------+---------+-------------+------------------+
|GST000001|  2024-12-27|Executive|       586.94|                10|
|GST000001|  2024-11-10|    Suite|      2441.59|                 6|
|GST000001|  2024-09-11| Standard|        97.25|                 7|
|GST000001|  2024-06-16| Standard|       983.68|                 8|
|GST000001|  2024-02-08|    Suite|       841.19|                 7|
+---------+------------+---------+-------------+------------------+



Records found: 5

=== Query 2: Recent High-Value Bookings ===


+------------+---------+---------+-------------+---------------+
|booking_date| guest_id|room_type|total_revenue|booking_channel|
+------------+---------+---------+-------------+---------------+
|  2024-10-16|GST002569|    Suite|      4518.02|         Direct|
|  2024-12-12|GST001814|    Suite|      4512.27|         Direct|
|  2024-11-08|GST004903|    Suite|      4473.05|         Direct|
|  2024-11-21|GST004845|    Suite|      4469.66|         Direct|
|  2024-12-12|GST002326|    Suite|      4420.99|         Direct|
|  2024-10-10|GST001027|    Suite|      4291.87|         Direct|
|  2024-11-30|GST002600|    Suite|      4260.58|        Walk-in|
|  2024-11-17|GST001894|    Suite|      4236.16|         Direct|
|  2024-10-04|GST004377|    Suite|      4214.49|        Walk-in|
|  2024-07-27|GST002825|    Suite|      4168.56|         Direct|
|  2024-12-28|GST002967|    Suite|      4163.38|         Direct|
|  2024-06-10|GST003431|    Suite|      4098.36|         Direct|
|  2024-07-04|GST000876| 

High-value bookings found: 6911

=== Query 3: Guest Spending Trends ===


+---------+------------+---------+-------------+------------------+
| guest_id|booking_date|room_type|total_revenue|guest_satisfaction|
+---------+------------+---------+-------------+------------------+
|GST000001|  2024-06-16| Standard|       983.68|                 8|
|GST000001|  2024-09-11| Standard|        97.25|                 7|
|GST000001|  2024-11-10|    Suite|      2441.59|                 6|
|GST000001|  2024-12-27|Executive|       586.94|                10|
|GST000002|  2024-07-01|   Deluxe|       928.56|                10|
|GST000003|  2024-05-09| Standard|       690.72|                 9|
|GST000003|  2024-08-10| Standard|       119.51|                 5|
|GST000003|  2024-08-26|   Deluxe|       550.91|                 9|
|GST000004|  2024-04-16| Standard|       557.51|                 8|
|GST000004|  2024-06-17|   Deluxe|       730.58|                10|
|GST000005|  2024-04-21|Executive|       315.68|                10|
|GST000005|  2024-06-30|    Suite|      2723.72|

Spending trend records found: 3737


## Step 6: Analyze Clustering Effectiveness

### Understanding the Impact

Let's examine how liquid clustering has organized our data and analyze some aggregate statistics to demonstrate the hospitality insights possible with this optimized structure.

### Key Analytics

- **Guest loyalty patterns** and repeat booking analysis
- **Revenue performance** by room type and booking channel
- **Seasonal trends** and occupancy optimization
- **Guest satisfaction** and service quality metrics

In [ ]:
# Analyze clustering effectiveness and hospitality insights


# Guest loyalty analysis

print("=== Guest Loyalty Analysis ===")

guest_loyalty = spark.sql("""

SELECT guest_id, COUNT(*) as total_bookings,

       ROUND(SUM(total_revenue), 2) as total_spent,

       ROUND(AVG(total_revenue), 2) as avg_booking_value,

       ROUND(AVG(guest_satisfaction), 2) as avg_satisfaction,

       MAX(booking_date) as last_booking_date

FROM hospitality.analytics.guest_stays

GROUP BY guest_id

ORDER BY total_spent DESC

LIMIT 10

""")



guest_loyalty.show()


# Room type performance

print("\n=== Room Type Performance ===")

room_performance = spark.sql("""

SELECT room_type, COUNT(*) as total_bookings,

       ROUND(SUM(total_revenue), 2) as total_revenue,

       ROUND(AVG(total_revenue), 2) as avg_revenue_per_booking,

       ROUND(AVG(guest_satisfaction), 2) as avg_satisfaction,

       COUNT(DISTINCT guest_id) as unique_guests

FROM hospitality.analytics.guest_stays

GROUP BY room_type

ORDER BY total_revenue DESC

""")



room_performance.show()


# Booking channel analysis

print("\n=== Booking Channel Performance ===")

channel_analysis = spark.sql("""

SELECT booking_channel, COUNT(*) as total_bookings,

       ROUND(SUM(total_revenue), 2) as total_revenue,

       ROUND(AVG(total_revenue), 2) as avg_revenue,

       ROUND(AVG(guest_satisfaction), 2) as avg_satisfaction,

       COUNT(DISTINCT guest_id) as unique_guests

FROM hospitality.analytics.guest_stays

GROUP BY booking_channel

ORDER BY total_revenue DESC

""")



channel_analysis.show()


# Monthly revenue trends

print("\n=== Monthly Revenue Trends ===")

monthly_trends = spark.sql("""

SELECT DATE_FORMAT(booking_date, 'yyyy-MM') as month,

       COUNT(*) as total_bookings,

       ROUND(SUM(total_revenue), 2) as monthly_revenue,

       ROUND(AVG(total_revenue), 2) as avg_booking_value,

       ROUND(AVG(guest_satisfaction), 2) as avg_satisfaction,

       COUNT(DISTINCT guest_id) as unique_guests

FROM hospitality.analytics.guest_stays

GROUP BY DATE_FORMAT(booking_date, 'yyyy-MM')

ORDER BY month

""")



monthly_trends.show()

=== Guest Loyalty Analysis ===


+---------+--------------+-----------+-----------------+----------------+-----------------+
| guest_id|total_bookings|total_spent|avg_booking_value|avg_satisfaction|last_booking_date|
+---------+--------------+-----------+-----------------+----------------+-----------------+
|GST000291|             8|   15489.66|          1936.21|             7.5|       2024-11-28|
|GST001027|             7|   14781.78|          2111.68|            7.71|       2024-12-29|
|GST000705|             8|   14645.07|          1830.63|             7.0|       2024-12-03|
|GST001894|             7|   14147.55|          2021.08|            7.43|       2024-12-11|
|GST002089|             7|   14125.12|          2017.87|             9.0|       2024-10-23|
|GST003861|             8|   14003.91|          1750.49|            8.25|       2024-12-19|
|GST003563|             8|   13950.03|          1743.75|            7.13|       2024-12-17|
|GST004202|             8|   13918.78|          1739.85|            8.13|       

+---------+--------------+-------------+-----------------------+----------------+-------------+
|room_type|total_bookings|total_revenue|avg_revenue_per_booking|avg_satisfaction|unique_guests|
+---------+--------------+-------------+-----------------------+----------------+-------------+
|    Suite|          6197|   9648746.75|                 1557.0|            7.98|         3580|
|Executive|          6189|   7706532.62|                 1245.2|            8.01|         3573|
|   Deluxe|          6273|   5616109.29|                 895.28|             8.0|         3632|
| Standard|          6266|   3365445.72|                  537.1|             7.0|         3609|
+---------+--------------+-------------+-----------------------+----------------+-------------+


=== Booking Channel Performance ===


+--------------------+--------------+-------------+-----------+----------------+-------------+
|     booking_channel|total_bookings|total_revenue|avg_revenue|avg_satisfaction|unique_guests|
+--------------------+--------------+-------------+-----------+----------------+-------------+
|              Direct|          6255|    7150741.3|     1143.2|            7.76|         3593|
|             Walk-in|          6131|   6663099.09|    1086.79|            7.73|         3589|
|           Corporate|          6307|   6460978.61|    1024.41|            7.78|         3608|
|Online Travel Agency|          6232|   6062015.38|     972.72|            7.72|         3580|
+--------------------+--------------+-------------+-----------+----------------+-------------+


=== Monthly Revenue Trends ===


+-------+--------------+---------------+-----------------+----------------+-------------+
|  month|total_bookings|monthly_revenue|avg_booking_value|avg_satisfaction|unique_guests|
+-------+--------------+---------------+-----------------+----------------+-------------+
|2024-01|          2047|     1889847.05|           923.23|            7.75|         1702|
|2024-02|          1980|     1826907.88|           922.68|            7.65|         1635|
|2024-03|          2136|     2001170.78|           936.88|            7.73|         1731|
|2024-04|          2051|     1888653.78|           920.85|            7.79|         1667|
|2024-05|          2078|     2163349.34|          1041.07|            7.76|         1723|
|2024-06|          2103|     2520292.68|          1198.43|            7.73|         1724|
|2024-07|          2085|     2492578.66|          1195.48|            7.74|         1696|
|2024-08|          2118|     2234283.63|           1054.9|            7.79|         1746|
|2024-09| 

## Key Takeaways: Delta Liquid Clustering in AIDP

### What We Demonstrated

1. **Automatic Optimization**: Created a table with `CLUSTER BY (guest_id, booking_date)` and let Delta automatically optimize data layout

2. **Performance Benefits**: Queries on clustered columns (guest_id, booking_date) are significantly faster due to data locality

3. **Zero Maintenance**: No manual partitioning, bucketing, or Z-Ordering required - Delta handles it automatically

4. **Real-World Use Case**: Hospitality analytics where guest experience and revenue management are critical

### AIDP Advantages

- **Unified Analytics**: Seamlessly integrates with other AIDP services
- **Governance**: Catalog and schema isolation for hospitality data
- **Performance**: Optimized for both OLAP and OLTP workloads
- **Scalability**: Handles hospitality-scale data volumes effortlessly

### Best Practices for Liquid Clustering

1. **Choose clustering columns** based on your most common query patterns
2. **Start with 1-4 columns** - too many can reduce effectiveness
3. **Consider cardinality** - high-cardinality columns work best
4. **Monitor and adjust** as query patterns evolve

### Next Steps

- Explore other AIDP features like AI/ML integration
- Try liquid clustering with different column combinations
- Scale up to larger hospitality datasets
- Integrate with real PMS systems and booking platforms

This notebook demonstrates how Oracle AI Data Platform makes advanced hospitality analytics accessible while maintaining enterprise-grade performance and governance.